# Fine-tuning a model with the Trainer API

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [25]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [1]:
from huggingface_hub import login
login()

In [4]:
!pip install fsspec==2023.6.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer", report_to = "none")

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [16]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,


)

In [17]:
trainer.train()

Step,Training Loss
500,0.515400
1000,0.269500


TrainOutput(global_step=1377, training_loss=0.3236828639876089, metrics={'train_runtime': 217.8382, 'train_samples_per_second': 50.515, 'train_steps_per_second': 6.321, 'total_flos': 405114969714960.0, 'train_loss': 0.3236828639876089, 'epoch': 3.0})

In [18]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [22]:
print(predictions.predictions)

[[-3.2420933   3.5523887 ]
 [ 2.698988   -2.790213  ]
 [-2.582269    2.9041824 ]
 [-2.9165854   3.2933962 ]
 [ 2.6231687  -2.6621552 ]
 [-2.7606354   3.1281312 ]
 [-2.668902    3.0144618 ]
 [-3.168563    3.534761  ]
 [-3.0061667   3.354326  ]
 [-3.1566844   3.491674  ]
 [-3.0927668   3.4208226 ]
 [ 2.0271897  -1.9067881 ]
 [ 2.8859556  -2.9771562 ]
 [-2.9487019   3.3041792 ]
 [-3.2469563   3.5222464 ]
 [-3.264205    3.5218637 ]
 [-3.2775927   3.546362  ]
 [ 1.871328   -1.9251459 ]
 [-3.2957978   3.5759554 ]
 [ 1.9718363  -1.8766335 ]
 [ 2.4379945  -2.3441997 ]
 [-2.8137648   3.1793556 ]
 [ 2.205291   -2.186555  ]
 [-3.13855     3.4730363 ]
 [-3.244824    3.524807  ]
 [-2.498741    2.8177416 ]
 [-1.3037031   1.3385886 ]
 [-3.3483105   3.5940983 ]
 [-3.051438    3.3859224 ]
 [-3.2158394   3.498557  ]
 [ 1.7545459  -1.8145343 ]
 [-3.23174     3.587296  ]
 [-2.918777    3.273558  ]
 [-2.875811    3.2783053 ]
 [-3.160567    3.4631126 ]
 [-2.7271366   3.0509977 ]
 [ 2.2025645  -2.0744777 ]
 

In [23]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [26]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()